<a href="https://colab.research.google.com/github/anelglvz/ML-AI-for-the-Working-Analyst/blob/main/Clasificador_de_Textos_con_Regresi%C3%B3n_log%C3%ADstica_usando_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error


nltk.download('stopwords')

In [ ]:
datos =datos=pd.read_csv('https://raw.githubusercontent.com/AnIsAsPe/ElementosEstadisticaMachineLearningPython/master/Semana8/spam_ham_dataset.csv',
                  usecols=['text','label_num'] )
datos.columns=['text','spam']

print(datos.shape)
datos.head()

In [ ]:
datos['spam'].value_counts()

## Preprocesamiento de textos

In [ ]:
def preprocesar(texto):
  #convierte a minúsculas
  texto = (texto).lower()

  #elimina stopwords + 'subject'
  stop = re.compile(r'\b(' + r'|'.join(stopwords.words('english')+(['subject'])) + r')\b\s*')
  texto = stop.sub('', texto) 

  #quita puntuaciones y números
  texto = re.sub('[^a-z]+', ' ', texto)

  #lematizar y quedarnos con palabras que tengan más de tres caracteres
  st = PorterStemmer()
  texto = texto.split()
  texto = ' '.join([st.stem(i) for i in texto])
  
  return(texto)

In [ ]:
datos['text_pp'] = datos['text'].apply(preprocesar)
datos

# Vectorización de Texto mediante BOW

In [ ]:
tfidf = TfidfVectorizer(min_df=3)
mdt_tfidf = tfidf.fit_transform(datos['text_pp']) 
palabras = tfidf.get_feature_names()
MDT = pd.DataFrame(mdt_tfidf.todense(), 
                              index=datos.index, 
                              columns=palabras)
print(MDT.shape)
MDT

# Dividir conjunto de entrenamiento y prueba

In [ ]:
X = mdt_tfidf.todense()
y = datos['spam']


In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2,  
                                                     random_state=3)

In [ ]:
#Conjunto de entrenamiento
X_train.shape, y_train.shape

In [ ]:
#Conjunto de prueba
X_test.shape, y_test.shape

# Diseño del modelo de red neuronal usando TensorFlow

In [ ]:
import tensorflow as tf  
print(tf.__version__)

In [ ]:
tf.random.set_seed(8)



TensorFlow y Keras, ambos proyectos para el aprendizaje profundo,tienen una historia entrelazada. Keras era un conjunto de abstracciones que facilita el aprendizaje profundo, pero necesitada de un backend, desde la versión de Keras v1.1.0 TensorFlow fue el backend predeterminado (antes era Theano).

A partir del lanzamiento de TensorFlow a mediados de 2019, Keras es ahora la API de alto nivel de TensorFlow para facilitar el diseño y entrenamiento de modelos rápidos y fáciles.  

[Video sobre TensorFlow 2.0](https://www.youtube.com/watch?v=EqWsPO8DVXk)

In [ ]:
model = tf.keras.Sequential([                     # https://www.tensorflow.org/guide/keras/sequential_model
                             
        tf.keras.Input(shape=(X.shape[1],)),              # El número de neuronas en la capa de entrada es igual al número de características o dimensiones en los datos. 
    
        tf.keras.layers.Dense(                       
                              1,                     # dimensiones de salida
                              activation='sigmoid',  # función de activación  https://www.tensorflow.org/api_docs/python/tf/keras/activations
                              name="layer_1"         # nombre de la capa

                              )
])
model.summary()

### Alternativas para inicializar una red Secuencial

Una alternativa para establecer las dimensiones de entrada en la primera capa es utilizar el parámetro `input_shape`

In [ ]:
model = tf.keras.Sequential([
    
          tf.keras.layers.Dense(1, 
                                input_shape = (X.shape[1],),      # dimensiones de la entrada  
                                activation='sigmoid',             # para la regresión logística
                                name="layer_1"                    # nombre de la capa
                                ),
])
model.summary()

Otra manera de construir un modelo secuencial es declararlo y a continuación añadir capas utilizando el método `add`

In [ ]:
model = tf.keras.Sequential()
    
model.add(tf.keras.Input(shape=(X.shape[1],)))

model.add(tf.keras.layers.Dense(1,
                                  activation='sigmoid',        # para la regresión logística
                                  name="layer_1"   
                                 ))
model.summary()

Que es lo mismo a:

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(1,
                                  input_shape = (X.shape[1],),
                                  activation='sigmoid',        # para la regresión logística
                                  name="layer_1"   
                                 ))
model.summary()

### Añadir y borrar capas, consultar número de capas de la red y  visualizar modelo,

In [ ]:
model.add(tf.keras.layers.Dense(1,
                                  activation='sigmoid',       
                                  name="layer_2")  
                                  )
model.summary()

In [ ]:
len(model.layers)  # layers es un atributo del modelo que regresa una lista con las capas del modelo

In [ ]:
tf.keras.utils.plot_model( 
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="LR",
    dpi=96,
)

Para borrar la última capa

In [ ]:

model.pop()    # borra la última capa
print(len(model.layers))
model.summary()


##<h2>Entrenamiento y validación

Despues de construir la red neuronal se debe configurar el proceso de entrenamiento utilizando el metodo `compile()`


In [ ]:
model.compile(optimizer='adam',             # el optimizador sirve para encontrar los pesos que minimizan la función de pérdida
                                            # adam: stochastic gradient descent adaptativo
                                            # https://keras.io/api/optimizers/adam/
              
              loss='binary_crossentropy',   # función que evalua que tan bien el algoritmo modela el conjunto de datos
                                            # https://keras.io/api/losses/
              
              metrics=['accuracy']
              )


Para entrenar el módelo utilizamos el método `fit()` 

el parámetro epoch se puede usar para buscar minimizar el error, aunque también podemos caer en sobreajuste

In [ ]:
model.fit( X_train, y_train,
          epochs=10,         # número de iteraciones sobre los datos (epocas)
          )

El 130/130 que vemos es el resultado redondeado hacia arriba, de dividir el número de registros en el conjunto de entrenamiento entre 32, que es el parámetro por defaul para batch_size

In [ ]:
# batch_size
X_train.shape[0]/32

# Red neuronal de clasificación con más de una capa

In [ ]:
model = tf.keras.Sequential()
    
model.add((tf.keras.layers.InputLayer(input_shape=(X.shape[1],))))

model.add(tf.keras.layers.Dense((2/3 * X.shape[1]  + 1 ), activation='relu',name="hidden_layer_1" ))

model.add(tf.keras.layers.Dense(1,activation='sigmoid', name="output_layer" ))



model.summary()

In [ ]:
print(10377*6918) #Parámetros para la primera capa
print(71788086+6918) #Sumamos los sesgos
print('Numero de parámetros:', model.count_params() ) #Solo se le suma el último sesgo

In [ ]:
tf.keras.utils.plot_model( 
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="LR",
    dpi=96,
)

In [ ]:
model.compile(optimizer='adam',             # stochastic gradient descent adaptativo
                                            # https://keras.io/api/optimizers/adam/
              
              loss='binary_crossentropy',   # función objetivo  que se busca minimizar
                                            # https://keras.io/api/losses/
              
              metrics=['accuracy']
              )
model.fit(X_train, y_train,
          epochs=5, 
          )

In [ ]:
model.predict(X_test)

In [ ]:
y_test

In [ ]:
#Obtenemos el MSE
mean_squared_error(model.predict(X_test),y_test)

# Referencia:

1. Keras vs. tf.keras: What’s the difference in TensorFlow 2.0? [Internet]. PyImageSearch. 2019 [citado 20 de julio de 2021]. Disponible en: https://www.pyimagesearch.com/2019/10/21/keras-vs-tf-keras-whats-the-difference-in-tensorflow-2-0/

1. Krishnan S. How to determine the number of layers and neurons in the hidden layer? [Internet]. Geek Culture. 2021 [citado 25 de abril de 2022]. Disponible en: https://medium.com/geekculture/introduction-to-neural-network-2f8b8221fbd3

